In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import calendar
import time


In [ ]:
'''Here where made a lot of test and even more failed than worked. I did not deleted all the trial and error'''

# Open the left and right video streams
left_video_path = 'test_data_for_depth/videos-2023-08-11 10_01_24.806689/leftCam.h265.mp4'
right_video_path = 'test_data_for_depth/videos-2023-08-11 10_01_24.806689/rightCam.h265.mp4'


cap_left = cv2.VideoCapture(left_video_path)
cap_right = cv2.VideoCapture(right_video_path)

if not cap_left.isOpened() or not cap_right.isOpened():
    print("Error: Could not open video files.")
    exit()

# Create a VideoWriter for saving the disparity view as a video
output_path = 'disparity_view.mp4'
frame_width = int(cap_left.get(3))
frame_height = int(cap_left.get(4))
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width * 2, frame_height))

# Create a StereoBM object for disparity calculation
stereo = cv2.StereoBM_create()

stereo.setNumDisparities(7 * 16)
stereo.setBlockSize(0 * 2 + 5)
stereo.setPreFilterType(1)
stereo.setPreFilterSize(7 * 2 + 5)
stereo.setPreFilterCap(50)
stereo.setTextureThreshold(40)
stereo.setUniquenessRatio(0)
stereo.setSpeckleRange(74)
stereo.setSpeckleWindowSize(0 * 2)
stereo.setDisp12MaxDiff(25)
stereo.setMinDisparity(25)


while True:
    # Read frames from the left and right video streams
    ret_left, frame_left = cap_left.read()
    ret_right, frame_right = cap_right.read()

    if not ret_left or not ret_right:
        break

    # Convert frames to grayscale (assuming they are originally in color)
    frame_left_gray = cv2.cvtColor(frame_left, cv2.COLOR_BGR2GRAY)
    frame_right_gray = cv2.cvtColor(frame_right, cv2.COLOR_BGR2GRAY)

    # Compute the disparity map
    disparity = stereo.compute(frame_left_gray, frame_right_gray)

    # Normalize the disparity map for better visualization
    disparity_normalized = cv2.normalize(disparity, dst=None, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX)
    disparity_8u = disparity_normalized.astype(np.uint8)

    # Apply the colormap to the CV_8UC1 disparity map
    disparity_color = cv2.applyColorMap(disparity_8u, cv2.COLORMAP_JET)

    # Overlap the left and right frames to create a disparity view
    overlap_image = cv2.addWeighted(frame_left, 0.5, frame_right, 0.5, 0)

    # Concatenate the original frames and the disparity map horizontally
    disparity_view = np.hstack((overlap_image, disparity_color))

    # Write the disparity view frame to the output video
    out.write(disparity_view)

    # Display the disparity view in real-time
    cv2.imshow('Disparity View', disparity_color)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture objects, the VideoWriter, and close the windows
cap_left.release()
cap_right.release()
out.release()
cv2.destroyAllWindows()


https://learnopencv.com/depth-perception-using-stereo-camera-python-c/

#Headmap offset to big
# Check for left and right camera IDs
# These values can change depending on the system
CamL_id = 2 # Camera ID for left camera
CamR_id = 0 # Camera ID for right camera

CamL= cv2.VideoCapture('test_data_for_depth/videos-2023-08-11 10_01_24.806689/leftCam.h265.mp4')
CamR= cv2.VideoCapture('test_data_for_depth/videos-2023-08-11 10_01_24.806689/rightCam.h265.mp4')

# Reading the mapping values for stereo image rectification
cv_file = cv2.FileStorage("data/stereo_rectify_maps.xml", cv2.FILE_STORAGE_READ)
Left_Stereo_Map_x = cv_file.getNode("Left_Stereo_Map_x").mat()
Left_Stereo_Map_y = cv_file.getNode("Left_Stereo_Map_y").mat()
Right_Stereo_Map_x = cv_file.getNode("Right_Stereo_Map_x").mat()
Right_Stereo_Map_y = cv_file.getNode("Right_Stereo_Map_y").mat()
cv_file.release()

def nothing(x):
    pass

cv2.namedWindow('disp',cv2.WINDOW_NORMAL)
cv2.resizeWindow('disp',600,600)

cv2.createTrackbar('numDisparities','disp',1,17,nothing)
cv2.createTrackbar('blockSize','disp',5,50,nothing)
cv2.createTrackbar('preFilterType','disp',1,1,nothing)
cv2.createTrackbar('preFilterSize','disp',2,25,nothing)
cv2.createTrackbar('preFilterCap','disp',5,62,nothing)
cv2.createTrackbar('textureThreshold','disp',10,100,nothing)
cv2.createTrackbar('uniquenessRatio','disp',15,100,nothing)
cv2.createTrackbar('speckleRange','disp',0,100,nothing)
cv2.createTrackbar('speckleWindowSize','disp',3,25,nothing)
cv2.createTrackbar('disp12MaxDiff','disp',5,25,nothing)
cv2.createTrackbar('minDisparity','disp',5,25,nothing)

# Creating an object of StereoBM algorithm
stereo = cv2.StereoBM_create()

while True:

  # Capturing and storing left and right camera images
  retL, imgL= CamL.read()
  retR, imgR= CamR.read()

  # Proceed only if the frames have been captured
  if retL and retR:
    imgR_gray = cv2.cvtColor(imgR,cv2.COLOR_BGR2GRAY)
    imgL_gray = cv2.cvtColor(imgL,cv2.COLOR_BGR2GRAY)

    # Applying stereo image rectification on the left image
    Left_nice= cv2.remap(imgL_gray,
              Left_Stereo_Map_x,
              Left_Stereo_Map_y,
              cv2.INTER_LANCZOS4,
              cv2.BORDER_CONSTANT,
              0)

    # Applying stereo image rectification on the right image
    Right_nice= cv2.remap(imgR_gray,
              Right_Stereo_Map_x,
              Right_Stereo_Map_y,
              cv2.INTER_LANCZOS4,
              cv2.BORDER_CONSTANT,
              0)

    # Updating the parameters based on the trackbar positions
    numDisparities = cv2.getTrackbarPos('numDisparities','disp')*16
    blockSize = cv2.getTrackbarPos('blockSize','disp')*2 + 5
    preFilterType = cv2.getTrackbarPos('preFilterType','disp')
    preFilterSize = cv2.getTrackbarPos('preFilterSize','disp')*2 + 5
    preFilterCap = cv2.getTrackbarPos('preFilterCap','disp')
    textureThreshold = cv2.getTrackbarPos('textureThreshold','disp')
    uniquenessRatio = cv2.getTrackbarPos('uniquenessRatio','disp')
    speckleRange = cv2.getTrackbarPos('speckleRange','disp')
    speckleWindowSize = cv2.getTrackbarPos('speckleWindowSize','disp')*2
    disp12MaxDiff = cv2.getTrackbarPos('disp12MaxDiff','disp')
    minDisparity = cv2.getTrackbarPos('minDisparity','disp')

    # Setting the updated parameters before computing disparity map
    stereo.setNumDisparities(numDisparities)
    stereo.setBlockSize(blockSize)
    stereo.setPreFilterType(preFilterType)
    stereo.setPreFilterSize(preFilterSize)
    stereo.setPreFilterCap(preFilterCap)
    stereo.setTextureThreshold(textureThreshold)
    stereo.setUniquenessRatio(uniquenessRatio)
    stereo.setSpeckleRange(speckleRange)
    stereo.setSpeckleWindowSize(speckleWindowSize)
    stereo.setDisp12MaxDiff(disp12MaxDiff)
    stereo.setMinDisparity(minDisparity)

    # Calculating disparity using the StereoBM algorithm
    disparity = stereo.compute(Left_nice,Right_nice)
    # NOTE: Code returns a 16bit signed single channel image,
    # CV_16S containing a disparity map scaled by 16. Hence it
    # is essential to convert it to CV_32F and scale it down 16 times.

    # Converting to float32
    disparity = disparity.astype(np.float32)

    # Scaling down the disparity values and normalizing them
    disparity = (disparity/16.0 - minDisparity)/numDisparities

    # Displaying the disparity map
    cv2.imshow("disp",disparity)

    # Close window using esc key
    if cv2.waitKey(1) == 27:
      break

  else:
    CamL= cv2.VideoCapture(CamL_id)
    CamR= cv2.VideoCapture(CamR_id)

import cv2

# Open the left and right video streams
left_video_path = 'test_data_for_depth/videos-2023-08-11 10_01_24.806689/leftCam.h265.mp4'
right_video_path = 'test_data_for_depth/videos-2023-08-11 10_01_24.806689/rightCam.h265.mp4'

cap_left = cv2.VideoCapture(left_video_path)
cap_right = cv2.VideoCapture(right_video_path)

if not cap_left.isOpened() or not cap_right.isOpened():
    print("Error: Could not open video files.")
    exit()

# Get the frame width and height
frame_width = int(cap_left.get(3))
frame_height = int(cap_left.get(4))

# Specify the horizontal shift amount in pixels
horizontal_shift = 100  # Adjust this value as needed

while True:
    # Read frames from the left and right video streams
    ret_left, frame_left = cap_left.read()
    ret_right, frame_right = cap_right.read()

    if not ret_left or not ret_right:
        break

    # Shift the frame of the right video horizontally
    if horizontal_shift > 0:
        frame_right_shifted = frame_right[:, horizontal_shift:]
    elif horizontal_shift < 0:
        frame_right_shifted = frame_right[:, :horizontal_shift]
    else:
        frame_right_shifted = frame_right

    # Resize frames if necessary (optional)
    # frame_left = cv2.resize(frame_left, (640, 480))
    # frame_right_shifted = cv2.resize(frame_right_shifted, (640, 480))

    # Display the left and right frames side by side as a stereo view
    stereo_view = cv2.hconcat([frame_left, frame_right_shifted])

    # Show the stereo view in the window
    cv2.imshow("Stereo View", stereo_view)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture objects and close the window
cap_left.release()
cap_right.release()
cv2.destroyAllWindows()


#Left and write together
# Capture video from left and right cameras (adjust the file paths)
left_video = cv2.VideoCapture('test_data_for_depth/videos-2023-08-11 10_01_24.806689/leftCam.h265.mp4')  # Adjust the index or file path as needed
right_video = cv2.VideoCapture('test_data_for_depth/videos-2023-08-11 10_01_24.806689/rightCam.h265.mp4')

# Lists to store calibration image points and object points for both cameras
left_image_points = []
right_image_points = []
object_points = []

# Define the checkerboard pattern size (e.g., 9x6)
pattern_size = (9, 6)

# Generate a grid of 3D points for the calibration target
objp = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:pattern_size[0], 0:pattern_size[1]].T.reshape(-1, 2)

while True:
    ret1, left_frame = left_video.read()
    ret2, right_frame = right_video.read()

    if not ret1 or not ret2:
        break

    # Convert frames to grayscale
    left_gray = cv2.cvtColor(left_frame, cv2.COLOR_BGR2GRAY)
    right_gray = cv2.cvtColor(right_frame, cv2.COLOR_BGR2GRAY)

    # Find checkerboard corners in both frames
    left_ret, left_corners = cv2.findChessboardCorners(left_gray, pattern_size, None)
    right_ret, right_corners = cv2.findChessboardCorners(right_gray, pattern_size, None)

    if left_ret and right_ret:
        # Store detected image points and object points
        left_image_points.append(left_corners)
        right_image_points.append(right_corners)
        object_points.append(objp)

# Perform camera calibration for left and right cameras
left_ret, left_mtx, left_dist, _, _ = cv2.calibrateCamera(object_points, left_image_points, left_gray.shape[::-1], None, None)
right_ret, right_mtx, right_dist, _, _ = cv2.calibrateCamera(object_points, right_image_points, right_gray.shape[::-1], None, None)

# Perform stereo calibration
stereo_ret, _, _, _, _, R, T, E, F = cv2.stereoCalibrate(
    object_points, left_image_points, right_image_points, left_mtx, left_dist, right_mtx, right_dist,
    left_gray.shape[::-1]
)

# Rectify the cameras using stereoRectify and initUndistortRectifyMap
# Use R, T, E, and F obtained from stereo calibration
# Capture frames from your stereo cameras and proceed with stereo vision processing


from matplotlib.pyplot import hsv

cap_right =cv2.VideoCapture('depthVision/test_data_for_depth/videos-2023-08-11 10_01_24.806689/rightCam.h265.mp4')
cap_left =cv2.VideoCapture('depthVision/test_data_for_depth/videos-2023-08-11 10_01_24.806689/leftCam.h265.mp4')

frame_rate = 30
B = 9
f = 6
alpha = 56.6

count = -1

while(True):
    count += 1

    ret_right, frame_right = cap_right.read()
    ret_left, frame_left = cap_left.read()

    if ret_right==False or ret_left==False:
        break
    else:
        mask_right = hsv.add_HSV_filter(frame_right, 1)

def videoToImageConverter(source_path: str, output_path: str, name: str):
    vidcap = cv2.VideoCapture(source_path)
    os.makedirs(output_path, exist_ok=True)
    count = 0

    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(vidcap.get(cv2.CAP_PROP_FPS))

    while count < total_frames:
        success, image = vidcap.read()

        if count % int(fps) == 0:
            if not image is None and not image.size == 0:
                cv2.imwrite(f"{output_path}/{name}_{count}.png", image)  # save frame as PNG file
            else:
                print(f"Skipping empty frame at count {count}")

        count += 1


    vidcap.release()
    cv2.destroyAllWindows()

cap = cv2.VideoCapture('test_data_for_depth/videos-2023-08-11 10_01_24.806689/leftCam.h265.mp4')

import os
import subprocess

FRAMERATE = 25



# Function to convert H.265 to MP4
def convert_h265_to_mp4(input_file):
    input_file = os.path.abspath(input_file)
    print(input_file)
    output_file = input_file + ".mp4"
    cmd = [
        "ffmpeg",
        "-framerate", str(FRAMERATE),
        "-i", input_file,
        "-c", "copy",
        output_file
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"Converted {input_file} to {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error converting {input_file}: {e}")

FRAMERATE = 25

# Function to convert H.265 to MP4
def convert_h265_to_mp4(input_file):

    input_file = os.path.abspath(input_file)


    output_file = input_file + ".mp4"
    cmd = [
        "ffmpeg",
        "-framerate", str(FRAMERATE),
        "-i", input_file,
        "-c", "copy",
        output_file
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"Converted {input_file} to {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error converting {input_file}: {e}")

# Loop through directories and files
for root, dirs, files in os.walk("."):
    for file in files:
        if file.endswith(".h265"):
            file_path = os.path.join(root, file)
            convert_h265_to_mp4(file_path)


convert_h265_to_mp4('test_data_for_depth/videos-2023-08-11 10_01_24.806689/leftCam.h265')

cap = cv2.VideoCapture('test_data_for_depth/videos-2023-08-11 10_01_24.806689/leftCam.h265')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'X265')
out = cv2.VideoWriter('output.mp4',fourcc, 20.0, (640,480))

while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        out.write(frame)

        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

from time import sleep
import cv2
import numpy as np

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
#cap = cv2.VideoCapture('test_data_for_depth/videos-2023-08-11 10_01_24.806689/leftCam.h265')


cap = cv2.VideoCapture('./test_data_for_depth/videos-2023-08-11 10_01_24.806689/leftCam.h265')

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_video.mp4', fourcc, 30, (int(cap.get(3)), int(cap.get(4))))

total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)

# Check if camera opened successfully
if (cap.isOpened()== False):
  print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    # Display the resulting frame
    cv2.imshow('Frame',frame)

    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else:
    break

# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

import numpy as np
import cv2

# Pfade zu den Videodateien für die linke und rechte Kamera
left_video_path = 'D:\\BigBlock_01\\tt_video_data\\videos-2023-09-18 14_53_22.084271\\leftCam.h265.mp4'
right_video_path = 'D:\\BigBlock_01\\tt_video_data\\videos-2023-09-18 14_53_22.084271\\rightCam.h265.mp4'


# Öffnen der Videodateien
CamL = cv2.VideoCapture(left_video_path)
CamR = cv2.VideoCapture(right_video_path)

# Pfad zur Ausgabedatei für die Disparitätskarte
output_path = 'disparity_view.mp4'

# Breite und Höhe der Videoframes
frame_width = int(CamL.get(3))
frame_height = int(CamL.get(4))

# Erstellen des VideoWriters für die Ausgabedatei
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width * 2, frame_height))

# Lesen der Mapping-Werte für die stereo Bildentzerrung
cv_file = cv2.FileStorage("data/stereo_rectify_maps.xml", cv2.FILE_STORAGE_READ)
Left_Stereo_Map_x = cv_file.getNode("Left_Stereo_Map_x").mat()
Left_Stereo_Map_y = cv_file.getNode("Left_Stereo_Map_y").mat()
Right_Stereo_Map_x = cv_file.getNode("Right_Stereo_Map_x").mat()
Right_Stereo_Map_y = cv_file.getNode("Right_Stereo_Map_y").mat()
cv_file.release()

# Erstellen eines StereoBM-Objekts
stereo = cv2.StereoBM_create()

while True:
    # Erfassen und Speichern der Bilder von linker und rechter Kamera
    retL, imgL = CamL.read()
    retR, imgR = CamR.read()

    if not retL or not retR:
        break

    # Umwandeln in Graustufen
    imgR_gray = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    imgL_gray = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)

    # Anwenden der stereo Bildentzerrung auf das linke Bild
    Left_nice = cv2.remap(imgL_gray,
                          Left_Stereo_Map_x,
                          Left_Stereo_Map_y,
                          cv2.INTER_LANCZOS4,
                          cv2.BORDER_CONSTANT,
                          0)

    # Anwenden der stereo Bildentzerrung auf das rechte Bild
    Right_nice = cv2.remap(imgR_gray,
                           Right_Stereo_Map_x,
                           Right_Stereo_Map_y,
                           cv2.INTER_LANCZOS4,
                           cv2.BORDER_CONSTANT,
                           0)

    # Setzen der Parameter für die Berechnung der Disparitätskarte
    stereo.setNumDisparities(7 * 16)
    stereo.setBlockSize(0 * 2 + 5)
    stereo.setPreFilterType(1)
    stereo.setPreFilterSize(7 * 2 + 5)
    stereo.setPreFilterCap(50)
    stereo.setTextureThreshold(40)
    stereo.setUniquenessRatio(0)
    stereo.setSpeckleRange(74)
    stereo.setSpeckleWindowSize(0 * 2)
    stereo.setDisp12MaxDiff(25)
    stereo.setMinDisparity(25)

    # Berechnung der Disparitätskarte
    disparity = stereo.compute(Left_nice, Right_nice)

    # Normalisieren und Skalieren der Disparitätskarte für die Darstellung
    disparity_normalized = cv2.normalize(disparity, dst=None, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX)
    disparity_8u = disparity_normalized.astype(np.uint8)

    # Anwenden einer Colormap auf die CV_8UC1 Disparitätskarte
    disparity_color = cv2.applyColorMap(disparity_8u, cv2.COLORMAP_JET)

    disparity = disparity_color.astype(np.float32)

    # Skalieren auf den Bereich [0, 1]
    disparity = (disparity / 255.0)

    # Schreiben des Frames in die Ausgabedatei
    out_frame = np.hstack((imgL, disparity_color))
    out.write(disparity)
    cv2.imshow("disp", disparity)

# Freigeben der VideoWriter-Ressourcen
out.release()

# Freigeben der VideoCapture-Ressourcen
CamL.release()
CamR.release()

# Zerstören aller offenen Fenster
cv2.destroyAllWindows()

print("Disparity-View Video wurde erfolgreich erstellt.")


import numpy as np
import cv2

# Pfade zu den Videodateien für die linke und rechte Kamera
left_video_path = 'data/videos-2023-08-11 10_01_24.806689/leftCam.h265.mp4'
right_video_path = 'data/videos-2023-08-11 10_01_24.806689/rightCam.h265.mp4'

# Öffnen der Videodateien
CamL = cv2.VideoCapture(left_video_path)
CamR = cv2.VideoCapture(right_video_path)

# Pfad zur Ausgabedatei für die Disparitätskarte
#
output_path = 'disparity_view.mp4'

# Breite und Höhe der Videoframes
#
frame_width = int(CamL.get(3))
#
frame_height = int(CamL.get(4))

# Create a VideoWriter for the output video
#
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))


while True:
    # Capture and store frames from the left and right cameras
    retL, imgL = CamL.read()
    retR, imgR = CamR.read()

    if not retL or not retR:
        break

    # Convert frames to grayscale
    imgR_gray = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY)
    imgL_gray = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY)

    # Apply stereo image rectification to the left image
    Left_nice = cv2.remap(imgL_gray,
                          Left_Stereo_Map_x,
                          Left_Stereo_Map_y,
                          cv2.INTER_LANCZOS4,
                          cv2.BORDER_CONSTANT,
                          0)

    # Apply stereo image rectification to the right image
    Right_nice = cv2.remap(imgR_gray,
                           Right_Stereo_Map_x,
                           Right_Stereo_Map_y,
                           cv2.INTER_LANCZOS4,
                           cv2.BORDER_CONSTANT,
                           0)

    # Set the parameters for computing the disparity map
    # ... (Previous code)

    # Compute the disparity map
    disparity = stereo.compute(Left_nice, Right_nice)

    # Normalize and scale the disparity map for visualization
    disparity_normalized = cv2.normalize(disparity, dst=None, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX)
    disparity_8u = disparity_normalized.astype(np.uint8)

    # Apply a colormap to the CV_8UC1 disparity map
    disparity_color = cv2.applyColorMap(disparity_8u, cv2.COLORMAP_JET)

    # Convert to float32 and scale to [0, 1]
    disparity = disparity_color.astype(np.float32) / 255.0

    # Write the disparity map to the output video
    out.write(disparity)

    # Display the disparity map
    cv2.imshow("Disparity Map", disparity)

    # Close the window using the 'Esc' key
    if cv2.waitKey(1) == 27:
        break

# Release VideoWriter and VideoCapture resources
out.release()
CamL.release()
CamR.release()

# Close all open windows
cv2.destroyAllWindows()

print("Disparity-View Video has been successfully created.")
